# Idiot
https://habrahabr.ru/post/342738/

In [2]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

In [3]:
with open('data/dostoevsky-all.txt', 'r') as f:
    text=f.read()
text = text.replace(u'\xa0', u' ').replace(u'\ufeff','') 
vocab = sorted(set(text))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [6]:
print(text[:300])
text[:300]

Федор Михайлович Достоевский
Бедные люди
Ох уж эти мне сказочники! Нет чтобы написать что-нибудь полезное, приятное, усладительное, а то всю подноготную в земле вырывают!.. Вот уж запретил бы им писать! Ну, на что это похоже: читаешь… невольно задумываешься, — а там всякая дребедень и пойдет в голов


'Федор Михайлович Достоевский\nБедные люди\nОх уж эти мне сказочники! Нет чтобы написать что-нибудь полезное, приятное, усладительное, а то всю подноготную в земле вырывают!.. Вот уж запретил бы им писать! Ну, на что это похоже: читаешь… невольно задумываешься, — а там всякая дребедень и пойдет в голов'

In [7]:
encoded[:110]

array([121, 138, 137, 147, 149,   1, 113, 141, 154, 133, 142, 144, 147,
       135, 141, 156,   1, 105, 147, 150, 151, 147, 138, 135, 150, 143,
       141, 142,   0, 102, 138, 137, 146, 160, 138,   1, 144, 163, 137,
       141,   0, 115, 154,   1, 152, 139,   1, 162, 151, 141,   1, 145,
       146, 138,   1, 150, 143, 133, 140, 147, 156, 146, 141, 143, 141,
         2,   1, 114, 138, 151,   1, 156, 151, 147, 134, 160,   1, 146,
       133, 148, 141, 150, 133, 151, 161,   1, 156, 151, 147,  10, 146,
       141, 134, 152, 137, 161,   1, 148, 147, 144, 138, 140, 146, 147,
       138,   9,   1, 148, 149, 141], dtype=int32)

In [8]:
len(vocab)

175

In [9]:
def get_batches(arr, n_seqs, n_steps):
    '''Создаем генератор, который возвращает пакеты размером
       n_seqs x n_steps из массива arr.
       
       Аргументы
       ---------
       arr: Массив, из которого получаем пакеты
       n_seqs: Batch size, количество последовательностей в пакете
       n_steps: Sequence length, сколько "шагов" делаем в пакете
    '''
    # Считаем количество символов на пакет и количество пакетов, которое можем сформировать
    characters_per_batch = n_seqs * n_steps
    n_batches = len(arr)//characters_per_batch
    
    # Сохраняем в массиве только символы, которые позволяют сформировать целое число пакетов
    arr = arr[:n_batches * characters_per_batch]
    
    # Делаем reshape 1D -> 2D, используя n_seqs как число строк, как на картинке
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        # пакет данных, который будет подаваться на вход сети
        x = arr[:, n:n+n_steps]
        # целевой пакет, с которым будем сравнивать предсказание, получаем сдвиганием "x" на один символ вперед
        y = np.zeros_like(x)
        y[:, :-1], y[:, -1] = x[:, 1:], x[:, 0]
        yield x, y

In [10]:
batches = get_batches(encoded, 10, 50)
x, y = next(batches)
print('x\n', x[:5, :5])
print('\ny\n', y[:5, :5])

x
 [[121 138 137 147 149]
 [147 145 152   1 156]
 [149 141 138 154 133]
 [151 147 145   1 141]
 [145   1 140 133   1]]

y
 [[138 137 147 149   1]
 [145 152   1 156 151]
 [141 138 154 133 144]
 [147 145   1 141   1]
 [  1 140 133   1 136]]


In [11]:
def build_inputs(batch_size, num_steps):
    ''' Определяем placeholder'ы для входных, целевых данных, а также вероятности drop out
    
        Аргументы
        ---------
        batch_size: Batch size, количество последовательностей в пакете
        num_steps: Sequence length, сколько "шагов" делаем в пакете
        
    '''
    # Объявляем placeholder'ы
    inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
    targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
    
    # Placeholder для вероятности drop out
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, keep_prob

In [12]:
def build_lstm(lstm_size, num_layers, batch_size, keep_prob):
    ''' Строим LSTM ячейку.
    
        Аргументы
        ---------
        keep_prob: Скаляр (tf.placeholder) для dropout keep probability
        lstm_size: Размер скрытых слоев в LSTM ячейках
        num_layers: Количество LSTM слоев
        batch_size: Batch size

    '''
    ### Строим LSTM ячейку
    
    def build_cell(lstm_size, keep_prob):
        # Начинаем с базовой LSTM ячейки
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        
        # Добавляем dropout к ячейке
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        return drop
    
    
    # Стэкируем несколько LSTM слоев для придания глубины нашему deep learning
    cell = tf.contrib.rnn.MultiRNNCell([build_cell(lstm_size, keep_prob) for _ in range(num_layers)])
    # Инициализируем начальное состояние LTSM ячейки
    initial_state = cell.zero_state(batch_size, tf.float32)
    
    return cell, initial_state

In [13]:
def build_output(lstm_output, in_size, out_size):
    ''' Строим softmax слой и возвращаем результат его работы.
    
        Аргументы
        ---------
        
        x: Входящий от LSTM тензор
        in_size: Размер входящего тензора, (кол-во LSTM юнитов скрытого слоя)
        out_size: Размер softmax слоя (объем словаря)
    
    '''

    # вытягиваем и решэйпим тензор, выполняя преобразование 3D -> 2D
    seq_output = tf.concat(lstm_output, axis=1)
    x = tf.reshape(seq_output, [-1, in_size])
    
    # Соединяем результат LTSM слоев с softmax слоем
    with tf.variable_scope('softmax'):
        softmax_w = tf.Variable(tf.truncated_normal((in_size, out_size), stddev=0.1))
        softmax_b = tf.Variable(tf.zeros(out_size))
    
    # Считаем logit-функцию
    logits = tf.matmul(x, softmax_w) + softmax_b
    # Используем функцию softmax для получения предсказания
    out = tf.nn.softmax(logits, name='predictions')
    
    return out, logits

In [14]:
def build_loss(logits, targets, lstm_size, num_classes):
    ''' Считаем функцию потери на основании значений logit-функции и целевых значений.
    
        Аргументы
        ---------
        logits: значение logit-функции
        targets: целевые значения, с которыми сравниваем предсказания
        lstm_size: Количество юнитов в LSTM слое
        num_classes: Количество классов в целевых значениях (размер словаря)
        
    '''
    # Делаем one-hot кодирование целевых значений и решейпим по образу и подобию logits
    y_one_hot = tf.one_hot(targets, num_classes)
    y_reshaped = tf.reshape(y_one_hot, logits.get_shape())
    
    # Считаем значение функции потери softmax cross entropy loss и возвращаем среднее значение
    loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped)
    loss = tf.reduce_mean(loss)
    return loss

In [15]:
def build_optimizer(loss, learning_rate, grad_clip):
    ''' Строим оптимизатор для обучения, используя обрезку градиента.
    
        Arguments:
        loss: значение функции потери
        learning_rate: параметр скорости обучения
    
    '''
    
    # Оптимизатор для обучения, обрезка градиента для контроля "взрывающихся" градиентов
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), grad_clip)
    train_op = tf.train.AdamOptimizer(learning_rate)
    optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    return optimizer

In [16]:
class CharRNN:
    
    def __init__(self, num_classes, batch_size=64, num_steps=50, 
                       lstm_size=128, num_layers=2, learning_rate=0.001, 
                       grad_clip=5, sampling=False):
    
        # Мы будем использовать эту же сеть для сэмплирования (генерации текста),
        # при этом будем подавать по одному символу за один раз
        if sampling == True:
            batch_size, num_steps = 1, 1
        else:
            batch_size, num_steps = batch_size, num_steps

        tf.reset_default_graph()
        
        # Получаем input placeholder'ы
        self.inputs, self.targets, self.keep_prob = build_inputs(batch_size, num_steps)

        # Строим LSTM ячейку
        cell, self.initial_state = build_lstm(lstm_size, num_layers, batch_size, self.keep_prob)

        ### Прогоняем данные через RNN слои
        # Делаем one-hot кодирование входящих данных
        x_one_hot = tf.one_hot(self.inputs, num_classes)
        
        # Прогоняем данные через RNN и собираем результаты
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=self.initial_state)
        self.final_state = state
        
        # Получаем предсказания (softmax) и результат logit-функции
        self.prediction, self.logits = build_output(outputs, lstm_size, num_classes)
        
        # Считаем потери и оптимизируем (с обрезкой градиента)
        self.loss = build_loss(self.logits, self.targets, lstm_size, num_classes)
        self.optimizer = build_optimizer(self.loss, learning_rate, grad_clip)

In [17]:
batch_size = 100        # Размер пакета
num_steps = 100         # Шагов в пакете
lstm_size = 512         # Количество LSTM юнитов в скрытом слое
num_layers = 2          # Количество LSTM слоев
learning_rate = 0.001   # Скорость обучения
keep_prob = 0.5         # Dropout keep probability

In [ ]:
epochs = 20
# Сохраняться каждый N итераций
save_every_n = 100

model = CharRNN(len(vocab), batch_size=batch_size, num_steps=num_steps,
                lstm_size=lstm_size, num_layers=num_layers, 
                learning_rate=learning_rate)

saver = tf.train.Saver(max_to_keep=100)
try:
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())

        # Можно раскомментировать строчку ниже и продолжить обучение с checkpoint'а
        saver.restore(sess, 'checkpoints/model.ckpt')
        counter = 0
        for e in range(epochs):
            # Обучаем сеть
            new_state = sess.run(model.initial_state)
            loss = 0
            for x, y in get_batches(encoded, batch_size, num_steps):
                counter += 1
                start = time.time()
                feed = {model.inputs: x,
                        model.targets: y,
                        model.keep_prob: keep_prob,
                        model.initial_state: new_state}
                batch_loss, new_state, _ = sess.run([model.loss, 
                                                     model.final_state, 
                                                     model.optimizer], 
                                                     feed_dict=feed)

                end = time.time()

                if (counter % save_every_n == 0):
                    saver.save(sess, "checkpoints/model.ckpt".format(counter, lstm_size))
                    print('Epoch: {}/{}... '.format(e+1, epochs),
                          'Training Step: {}... '.format(counter),
                          'Training loss: {:.4f}... '.format(batch_loss),
                          '{:.4f} sec/batch'.format((end-start)))
except KeyboardInterrupt:
    pass

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
Epoch: 1/20...  Training Step: 100...  Training loss: 1.3859...  0.1575 sec/batch
Epoch: 1/20...  Training Step: 200...  Training loss: 1.4410...  0.1764 sec/batch


In [19]:
def pick_top_n(preds, vocab_size, top_n):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [20]:
def sample(n_samples, lstm_size, vocab_size, top_n, prime="Глаза его были большие"):
    samples = [c for c in prime]
    model = CharRNN(len(vocab), lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, 'checkpoints/model.ckpt')
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab), top_n)
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.prediction, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab), top_n)
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [34]:
# samp = sample(1000, lstm_size, len(vocab), top_n=5, prime='Лев Николаевич нес уже полнейший бред:')
primes = ['Улюкаев был задержан с поличным', 'Президент Америки', 'Улюкаева могут освободить по состоянию здоровья']
for prime in primes:
    samp = sample(200, lstm_size, len(vocab), top_n=3, prime=prime)
    samp = samp.replace('\n', '\n\n').split('.')[0]+'.'
    print(samp)

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
Улюкаев был задержан с поличным советом, и всё это было, что он никто не поднялся и не понемал в темноте.
INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
Президент Америки.
INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
Улюкаева могут освободить по состоянию здоровья совершенно принесло всего вопроса и все эти пристальнее, что не помните вашей досаде.
